In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

import os
import importlib
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
import utils.text
import utils.api

importlib.reload(utils.text)
importlib.reload(utils.api)

from utils.text import match_misconception, clean_string
from utils.api import get_completion

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
misconceptions = pd.read_csv('eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv', index_col='MisconceptionId')
train = pd.read_csv('eedi-mining-misconceptions-in-mathematics/train.csv')
train_melted = pd.read_csv('train_melted.csv')
test = pd.read_csv('eedi-mining-misconceptions-in-mathematics/test.csv')

In [ ]:
misconception_list = list(misconceptions['MisconceptionName'])
misconception_list

In [ ]:
# Create subset of train where misconception is in misconceptions
train_subset = train_melted[train_melted['MisconceptionName'].notna()]
train_subset = train_subset.sample(n=5)
train_subset

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,QuestionText,AnswerText,AnswerOption,MisconceptionId,MisconceptionName
3198,1329,427,Add proper fractions with different denominators which do not share a common factor,230,Adding and Subtracting Fractions,A,\(\n\frac{1}{3}+\frac{1}{10}=\frac{\bigstar}{30}\n\)\nWhat should replace the star?,\( 2 \),B,907.0,Does not find a common denominator when adding/subtracting fractions
186,186,26,Express words as 5 digit numbers and above,202,Place Value,C,How do you write this number in digits?\nFifty-six thousand and thirteen,\( 5613 \),A,1526.0,Believes that they only need to write non-zero digits when giving a number in digits
2358,489,2580,"Given the area of a square, calculate a missing dimension",75,Area of Simple Shapes,C,A square has an area of \( 64 \mathrm{~cm}^{2} \) What is the length of the square?,\( 16 \mathrm{~cm} \),B,1678.0,Confuses perimeter and area
5044,1306,1382,"Given a midpoint and a coordinate work out the position of the other coordinate, where at least one value is negative and at least one is a non-integer",408,Midpoint Between Two Co-ordinates,B,"Here are two points and their midpoint.\nWhat should replace the triangle? ![The image is a set of axes with three points marked. One point is (1,7). The midpoint is (-1, 0.5). The third point is given as (star, triangle), so the triangle is the y value of the point.]()",\( \Delta=-6.5 \),C,2275.0,Believes midpoint calculation is just half of the difference
4136,398,3326,Count backwards through zero starting from a positive integer,211,Adding and Subtracting Negative Numbers,A,John's flat is on floor \( 4 \).\n\nThe gym is on floor \( -1 \).\n\nHow many floors does John need to go down to get from his flat to the gym? ![A vertical number line from -8 to 7 increasing in 1 digit increments](),\( 6 \),C,1824.0,Counts the starting number when counting backwards


In [ ]:
for index, row in train_subset.tail(1).iterrows():
    subject = row['SubjectName']
    question_text = row['QuestionText']
    answer_text = row['AnswerText']
    prompt = f"""Given the following question and answer option, complete the following tasks: 
1. Identify whether the answer is correct. 
2. If the answer is correct, simply output "Correct answer."
3. If incorrect, identify the misconception that would cause someone to arrive at the answer and just print that as your response.
Subject: {subject}
Question: {question_text}
Answer: {answer_text}"""
    
    response = match_misconception(get_completion(prompt, max_tokens=100), subject)

    print("-"*100)
    print("PROMPT")
    print(prompt)
    print("RESPONSE")
    print(response)
    print("ACTUAL")
    print(row['MisconceptionName'])



----------------------------------------------------------------------------------------------------
PROMPT
Given the following question and answer option, complete the following tasks: 
1. Identify whether the answer is correct. 
2. If the answer is correct, simply output "Correct answer."
3. If incorrect, identify the misconception that would cause someone to arrive at the answer and just print that as your response.
Subject: Adding and Subtracting Negative Numbers
Question: John's flat is on floor \( 4 \).

The gym is on floor \( -1 \).

How many floors does John need to go down to get from his flat to the gym? ![A vertical number line from -8 to 7 increasing in 1 digit increments]()
Answer: \( 6 \)
RESPONSE
"Does not know that angles on a straight line sum to 180 degrees."
ACTUAL
Counts the starting number when counting backwards
